# Tutorial for CLI tool `clkhash`

For this tutorial we are going to process a data set for private linkage with clkhash using the command line tool `clkutil` - equivalent to running `python -m clkhash`.

Note you can also use the [Python API](./tutorial_api.ipynb).

The Python package `recordlinkage` has a [tutorial](http://recordlinkage.readthedocs.io/en/latest/notebooks/link_two_dataframes.html) linking data sets in the clear, we will try duplicate that in a privacy preserving setting.

First install clkhash, recordlinkage and a few data science tools (pandas and numpy).

    $ pip install -U clkhash recordlinkage numpy pandas

In [ ]:
import json
import numpy as np
import pandas as pd

In [ ]:
import recordlinkage
from recordlinkage.datasets import load_febrl4

## Data Exploration

First we have a look at the dataset.

In [ ]:
dfA, dfB = load_febrl4()

dfA.head()

Note that for computing this linkage we will **not** use the social security id column or the `rec_id` index.

In [ ]:
dfA.columns

In [ ]:
dfA.to_csv('PII_a.csv')

## Hashing Schema Definition

A hashing schema instructs clkhash how to treat each column for generating CLKs. A detailed description of the hashing schema can be found in the [api docs](http://clkhash.readthedocs.io/en/latest/schema.html). We will ignore the columns 'rec_id' and 'soc_sec_id' for CLK generation.



In [ ]:
with open("_static/febrl_schema_v2_overweight.json") as f:
    print(f.read())

## Validate the schema

The command line tool can check that the linkage schema is valid:

In [ ]:
!clkutil validate-schema _static/febrl_schema_v2_overweight.json

## Hash the data

We can now hash our Personally Identifiable Information (PII) data from the CSV file using our defined linkage schema. We must provide two *secret keys* to this command - these keys have to be used by both parties hashing data. For this toy example we will use the keys _'key1'_ and _'key2'_, for real data, make sure that the keys contain enough entropy, as knowledge of these keys is sufficient to reconstruct the PII information from a CLK! Also, **do not share these keys with anyone, except the other participating party.**

In [ ]:
!clkutil hash PII_a.csv key1 key2 _static/febrl_schema_v2_overweight.json clks_a.json

## Inspect the output

clkhash has hashed the PII, creating a Cryptographic Longterm Key for each entity. The stats output shows that the mean popcount (number of bits set) is quite high (949 out of 1024) which can effect accuracy.

To reduce the popcount you can modify the individual _'numBits'_ values for the different fields. It allows to tune the contribution of a column to the CLK. This can be used to de-emphasise columns which are less suitable for linkage (e.g. information that changes frequently).

In [ ]:
!clkutil describe clks_a.json

First, we will reduce the value of *numBits* for each feature.

In [ ]:
with open("_static/febrl_schema_v2_reduced.json") as f:
    print(f.read())

In [ ]:
!clkutil hash PII_a.csv key1 key2 _static/febrl_schema_v2_reduced.json clks_a.json

And now we will modify the `numBits` values again, this time de-emphasising the contribution of the address related columns.

In [ ]:
with open("_static/febrl_schema_v2_final.json") as f:
    print(f.read())

In [ ]:
!clkutil hash PII_a.csv key1 key2 _static/febrl_schema_v2_final.json clks_a.json

Great, now approximately half the bits are set in each CLK. 

Each CLK is serialized in a JSON friendly base64 format:

In [ ]:
# If you have jq tool installed:
#!jq .clks[0] clks_a.json

import json
json.load(open('clks_a.json'))['clks'][0]

## Hash data set B

Now we hash the second dataset using the same keys and same schema.

In [ ]:
dfB.to_csv('PII_b.csv')

!clkutil hash PII_b.csv key1 key2 _static/febrl_schema_v2_final.json clks_b.json

## Find matches between the two sets of CLKs

We have generated two sets of CLKs which represent entity information in a privacy-preserving way. The more similar two CLKs are, the more likely it is that they represent the same entity.

For this task we will use the entity service, which is provided by Data61. 
The necessary steps are as follows:
- The analyst creates a new project with the output type 'mapping'. They will receive a set of credentials from the server.
- The analyst then distributes the `update_tokens` to the participating data providers.
- The data providers then individually upload their respective CLKs.
- The analyst can create *runs* with various thresholds (and other settings)
- After the entity service successfully computed the mapping, it can be accessed by providing the `result_token`

First we check the status of an entity service:

In [ ]:
SERVER = 'https://testing.es.data61.xyz'

!clkutil status --server={SERVER}

The analyst creates a new project on the entity service by providing the hashing schema and result type. The server returns a set of credentials which provide access to the further steps for project.

In [ ]:
!clkutil create-project --server={SERVER} --schema _static/febrl_schema_v2_final.json --output credentials.json --type "mapping" --name "tutorial"

The returned credentials contain a 
- `project_id`, which identifies the project
- `result_token`, which gives access to the mapping result, once computed
- `upload_tokens`, one for each provider, allows uploading CLKs.

In [ ]:
credentials = json.load(open('credentials.json', 'rt'))
print(json.dumps(credentials, indent=4))

### Uploading the CLKs to the entity service
Each party individually uploads its respective CLKs to the entity service. They need to provide the `resource_id`, which identifies the correct mapping, and an `update_token`.

In [ ]:
!clkutil upload \
       --project="{credentials['project_id']}" \
        --apikey="{credentials['update_tokens'][0]}" \
        --output "upload_a.json" \
        --server="{SERVER}" \
       "clks_a.json"

In [ ]:
!clkutil upload \
       --project="{credentials['project_id']}" \
        --apikey="{credentials['update_tokens'][1]}" \
        --output "upload_b.json" \
        --server="{SERVER}" \
       "clks_b.json"

Now that the CLK data has been uploaded the analyst can create one or more *runs*. Here we will start by calculating a mapping with a threshold of `0.9`:

In [ ]:
!clkutil create --verbose  \
    --server="{SERVER}" \
    --output "run_info.json" \
    --threshold=0.9 \
    --project="{credentials['project_id']}" \
    --apikey="{credentials['result_token']}" \
    --name="CLI tutorial run A"

In [ ]:
run_info = json.load(open('run_info.json', 'rt'))
run_info

## Results

Now after some delay (depending on the size) we can fetch the results. This can be done with clkutil:

In [ ]:
!clkutil results --watch \
        --project="{credentials['project_id']}" \
        --apikey="{credentials['result_token']}" \
        --run="{run_info['run_id']}" \
        --server="{SERVER}" \
        --output results.txt

In [ ]:
with open('results.txt') as f:
    str_mapping = json.load(f)['mapping']

mapping = {int(k): int(v) for k,v in str_mapping.items()}
print('The service linked {} entities.'.format(len(mapping)))

Let's investigate some of those matches and the overall matching quality. In this case we have the ground truth so we can compute the precision, recall, and accuracy.

In [ ]:
with open('PII_a.csv', 'rt') as f:
    a_raw = f.readlines()
with open('PII_b.csv', 'rt') as f:
    b_raw = f.readlines()

num_entities = len(b_raw) - 1

def describe_accuracy(mapping, show_examples=False):
    if show_examples:
        print('idx_a, idx_b,     rec_id_a,       rec_id_b')
        print('---------------------------------------------')
        for a_i in range(10):
            if a_i in mapping:
                a_data = a_raw[a_i + 1].split(',')
                b_data = b_raw[mapping[a_i] + 1].split(',')
                print('{:3}, {:6}, {:>15}, {:>15}'.format(a_i+1, mapping[a_i]+1, a_data[0], b_data[0]))
        print('---------------------------------------------')
    
    TP = 0; FP = 0; TN = 0; FN = 0
    for a_i in range(num_entities):
        if a_i in mapping:
            if a_raw[a_i + 1].split(',')[0].split('-')[1] == b_raw[mapping[a_i] + 1].split(',')[0].split('-')[1]:
                TP += 1
            else:
                FP += 1
                # as we only report one mapping for each element in PII_a, 
                # then a wrong mapping is not only a false positive, but 
                # also a false negative, as we won't report the true mapping.
                FN += 1 
        else:
            FN += 1 # every element in PII_a has a partner in PII_b


    print('Precision: {:.2f}, Recall: {:.2f}, Accuracy: {:.2f}'.format(
        TP/(TP+FP), 
        TP/(TP+FN), 
        (TP+TN)/(TP+TN+FP+FN)))

In [ ]:
describe_accuracy(mapping, True)

Precision tells us about how many of the found matches are actual matches. The score of 1.0 means that we did perfectly in this respect, however, **recall**, the measure of how many of the actual matches were correctly identified, is quite low with only 80%.

Let's go back and create another mapping with a `threshold` value of `0.8`.

In [ ]:
!clkutil create --verbose  \
    --server="{SERVER}" \
    --output "run_info.json" \
    --threshold=0.8 \
    --project="{credentials['project_id']}" \
    --apikey="{credentials['result_token']}" \
    --name="CLI tutorial run B"

run_info = json.load(open('run_info.json', 'rt'))

In [ ]:
!clkutil results --watch \
        --project="{credentials['project_id']}" \
        --apikey="{credentials['result_token']}" \
        --run="{run_info['run_id']}" \
        --server="{SERVER}" \
        --output results.txt

In [ ]:
with open('results.txt') as f:
    str_mapping = json.load(f)['mapping']

mapping = {int(k): int(v) for k,v in str_mapping.items()}

print('The service linked {} entities.'.format(len(mapping)))
describe_accuracy(mapping)

Great, for this threshold value we get a precision of 100% and a recall of 99%. 

The explanation is that when the information about an entity differs slightly in the two datasets (e.g. spelling errors, abbrevations, missing values, ...) then the corresponding CLKs will differ in some number of bits as well. For the datasets in this tutorial the perturbations are such that only 80% of the derived CLK pairs overlap more than 90% (the first threshold). Whereas 99% of all matching pairs overlap more than 80%.

If we keep reducing the threshold value, then we will start to observe mistakes in the found matches -- the precision decreases (if an entry in dataset A has no match in dataset B, but we keep reducing the threshold, eventually a comparison with an entry in B will be above the threshold leading to a false match). But at the same time the recall value will keep increasing for a while, as a lower threshold allows for more of the actual matches to be found. However, as our example dataset only contains matches (every entry in A has a match in B), this phenomenon cannot be observered. With the threshold `0.75` we identify all matches correctly (at the cost of a longer execution time).

In [ ]:
!clkutil create --verbose  \
    --server="{SERVER}" \
    --output "run_info.json" \
    --threshold=0.75 \
    --project="{credentials['project_id']}" \
    --apikey="{credentials['result_token']}" \
    --name="CLI tutorial run B"

run_info = json.load(open('run_info.json', 'rt'))

In [ ]:
!clkutil results --watch \
        --project="{credentials['project_id']}" \
        --apikey="{credentials['result_token']}" \
        --run="{run_info['run_id']}" \
        --server="{SERVER}" \
        --output results.txt

In [ ]:
with open('results.txt') as f:
    str_mapping = json.load(f)['mapping']

mapping = {int(k): int(v) for k,v in str_mapping.items()}

print('The service linked {} entities.'.format(len(mapping)))
describe_accuracy(mapping)

It is important to choose an appropriate threshold for the amount of perturbations present in the data.

Feel free to go back to the CLK generation and experiment on how different setting will affect the matching quality.

## Cleanup

Finally to remove the results from the service delete the individual runs, or remove the uploaded data and all runs by deleting the entire project.

In [ ]:
# Deleting a run
!clkutil delete --project="{credentials['project_id']}" \
        --apikey="{credentials['result_token']}" \
        --run="{run_info['run_id']}" \
        --server="{SERVER}"

In [ ]:
# Deleting a project
!clkutil delete-project --project="{credentials['project_id']}" \
        --apikey="{credentials['result_token']}" \
        --server="{SERVER}"